In [11]:
import csv
import json
import nltk
from nltk.translate.bleu_score import sentence_bleu
from translator_es_en import Translator

# Load the JSON files
with open('../data/xquad.en.json', 'r', encoding='utf-8') as file:
    data_en = json.load(file)

with open('../data/xquad.es.json', 'r', encoding='utf-8') as file:
    data_es = json.load(file)

# Assuming both JSON files are structured similarly and contain a list of dictionaries
# with 'context' and 'question' keys among others. Modify the keys based on your JSON structure.

# Initialize your Translator
translator = Translator("cpu")

# List to store BLEU scores
bleu_scores = []

# Open CSV file in append mode
with open('bleu_scores.csv', 'a', newline='') as csvfile:
  writer = csv.writer(csvfile)

# Loop through the datasets
for item_en, item_es in zip(data_en['data'], data_es['data']):
    for paragraph_en, paragraph_es in zip(item_en['paragraphs'], item_es['paragraphs']):
        # Extracting text; this assumes your JSON has these fields. Adjust if necessary.
        source_text = paragraph_es['context']  # Spanish text
        target_text = paragraph_en['context']  # English text

        # Translate Spanish to English
        translated_text = translator.translate(source_text)

        # Compute the BLEU score
        # NLTK expects a list of reference sentences, each as a list of tokens
        # and the candidate sentence as a list of tokens
        reference = [nltk.word_tokenize(target_text.lower())]
        candidate = nltk.word_tokenize(translated_text.lower())
        score = sentence_bleu(reference, candidate)
        
        # Write score to CSV on every iteration
        with open('bleu_scores.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([score])

# Print average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f'Average BLEU Score: {average_bleu}')
